# Simple retrieval augmented generation
In this notebook we see how retrieval augmented generation (RAG) works using OpenAI and numpy.

In [1]:
import numpy as np
import openai

We aim to answer this question:

In [2]:
question = "How can I label objects in an image?"

... using these code snippets (and more):

In [3]:
with open('code_snippets.txt', 'r') as file:
    all_code_snippets = file.read()

In [4]:
splits = all_code_snippets.split("\n\n")
_ = [print(s) for s in splits[:3]]

* Displays an image with a slider and label showing mouse position and intensity.
stackview.annotate(image, labels)
* Allows cropping an image along all axes.
stackview.crop(image)
* Showing an image stored in variable `image` and a segmented image stored in variable `labels` on top. Also works with two images or two label images.
stackview.curtain(image, labels, alpha: float = 1)


## Vector embeddings
To make our code snippets searchable, we need to embed them, we need to turn them into vectors.

In [5]:
def embed(text):
    from openai import OpenAI
    client = OpenAI()

    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [6]:
vector = embed("Hello world")
vector[:3], len(vector)

([-0.002142224693670869, -0.04909248650074005, 0.02102646231651306], 1536)

## Vector store
We also need a vector store, which is basically just a dictionary that allows us to quickly find a text given a corresponding vector, or a vector that has a short distance to it.

In [7]:
class VectorStore:
    def __init__(self, texts=None):
        self._store = {}
        if texts is not None:
            for text in texts:
                self._store[tuple(embed(text))] = text
    
    def search(self, text, n_best_results=3):
        single_vector = embed(text)
        
        # Step 1: Compute Euclidean distances
        distances = [(np.linalg.norm(np.asarray(single_vector) - np.asarray(vector)), vector) for vector in self._store.keys()]

        # Step 2: Sort distances and get the three vectors with the shortest distances
        distances.sort()  # Sort based on the first element in the tuple (distance)
        closest_vectors = [vec for _, vec in distances[:n_best_results]]  # Extract only the vectors

        self.distances = distances
        
        return [self._store[tuple(v)] for v in closest_vectors]
    
    def get_text(self, vector):
        return self._store[vector]

In [8]:
vectore_store = VectorStore(splits)

## Searching the vector store
We can then search in the store for vectors and corresponding texts that are close by a given question.

In [9]:
question

'How can I label objects in an image?'

In [10]:
code_snippets = vectore_store.search(question)
code_snippets

['* Labels objects in grey-value images using Gaussian blurs, spot detection, Otsu-thresholding, and Voronoi-labeling from isotropic input images.\ncle.voronoi_otsu_labeling(source: ndarray, label_image_destination: ndarray = None, spot_sigma: float = 2, outline_sigma: float = 2) -> ndarray',
 '* Apply morphological opening operation, fill label gaps with voronoi-labeling, and mask background pixels in label image.\ncle.smooth_labels(labels_input: ndarray, labels_destination: ndarray = None, radius: int = 0) -> ndarray',
 '* Dilates labels in an isotropic label image without overwriting other labels.\ncle.dilate_labels(labeling_source: ndarray, labeling_destination: ndarray = None, radius: int = 2) -> ndarray']

## Prompting OpenAI
We will also need access to a large language model (LLM) to combine the code snippets and the question to retrieve an answer to our question that involves the code snippets.

In [11]:
def prompt_chatGPT(message:str, model="gpt-3.5-turbo"):
    """A prompt helper function that sends a message to openAI
    and returns only the text response.
    """
    import os
    import openai
    
    # convert message in the right format if necessary
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
        
    # setup connection to the LLM
    # todo: enter your API key here:
    client = openai.OpenAI(api_key = os.environ.get('OPENAI_API_KEY'))
    
    # submit prompt
    response = client.chat.completions.create(
        model=model,
        messages=message
    )
    
    # extract answer
    return response.choices[0].message.content

We can then assemble code snippets and question to a prompt.

In [19]:
context = "\n\n".join(code_snippets)

prompt = f"""
Answer the question by the very end and take given code snippets into account.

## Code snippets
{context}

## Question
{question}
"""

print(prompt)


Answer the question by the very end and take given code snippets into account.

## Code snippets
* Labels objects in grey-value images using Gaussian blurs, spot detection, Otsu-thresholding, and Voronoi-labeling from isotropic input images.
cle.voronoi_otsu_labeling(source: ndarray, label_image_destination: ndarray = None, spot_sigma: float = 2, outline_sigma: float = 2) -> ndarray

* Apply morphological opening operation, fill label gaps with voronoi-labeling, and mask background pixels in label image.
cle.smooth_labels(labels_input: ndarray, labels_destination: ndarray = None, radius: int = 0) -> ndarray

* Dilates labels in an isotropic label image without overwriting other labels.
cle.dilate_labels(labeling_source: ndarray, labeling_destination: ndarray = None, radius: int = 2) -> ndarray

## Question
How can I label objects in an image?



## Answering our question
Eventually we can answer our question

In [20]:
answer = prompt_chatGPT(prompt)

print(answer)

You can label objects in an image by using the provided code snippets. First, you can use `cle.voronoi_otsu_labeling` to label objects in grey-value images using Gaussian blurs, spot detection, Otsu-thresholding, and Voronoi-labeling. Then, you can apply morphological operations and fill label gaps with voronoi-labeling using `cle.smooth_labels`. Finally, you can dilate the labels in the image without overwriting other labels using `cle.dilate_labels`. By combining these steps, you can effectively label objects in an image.
